### 🧠 What is Query Decomposition?
Query decomposition is the process of taking a complex, multi-part question and breaking it into simpler, atomic sub-questions that can each be retrieved and answered individually.

#### ✅ Why Use Query Decomposition?

- Complex queries often involve multiple concepts

- LLMs or retrievers may miss parts of the original question

- It enables multi-hop reasoning (answering in steps)

- Allows parallelism (especially in multi-agent frameworks)

In [1]:
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableSequence

In [2]:
# Step 1: Load and embed the document
loader = TextLoader("langchain_crewai_dataset.txt")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding)
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 4, "lambda_mult": 0.7})

/Users/saonpaul/Code/RAG_Practice/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=init_chat_model(model="groq:gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x31a196e40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x17dde2c00>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
# Step 3: Query decomposition
decomposition_prompt = PromptTemplate.from_template("""
You are an AI assistant. Decompose the following complex question into 2 to 4 smaller sub-questions for better document retrieval.

Question: "{question}"

Sub-questions:
""")
decomposition_chain = decomposition_prompt | llm | StrOutputParser()

In [5]:
query = "How does LangChain use memory and agents compared to CrewAI?"
decomposition_question=decomposition_chain.invoke({"question": query})


In [6]:
print(decomposition_question)

Here are some sub-questions that break down the complex query:

1. **What memory mechanisms does LangChain utilize?** 
2. **How do LangChain agents leverage memory?**
3. **What memory capabilities does CrewAI offer?**
4. **How do CrewAI's agents differ from LangChain agents in terms of memory usage?** 


These sub-questions target specific aspects of memory and agent functionality in both LangChain and CrewAI, allowing for more focused and precise document retrieval. 



In [7]:
# Step 4: QA chain per sub-question
qa_prompt = PromptTemplate.from_template("""
Use the context below to answer the question.

Context:
{context}

Question: {input}
""")
qa_chain = create_stuff_documents_chain(llm=llm, prompt=qa_prompt)

In [8]:
# Step 5: Full RAG pipeline logic
def full_query_decomposition_rag_pipeline(user_query):
    # Decompose the query
    sub_qs_text = decomposition_chain.invoke({"question": user_query})
    sub_questions = [q.strip("-•1234567890. ").strip() for q in sub_qs_text.split("\n") if q.strip()]
    
    results = []
    for subq in sub_questions:
        docs = retriever.invoke(subq)
        result = qa_chain.invoke({"input": subq, "context": docs})
        results.append(f"Q: {subq}\nA: {result}")
    
    return "\n\n".join(results)

In [9]:
# Step 6: Run
query = "How does LangChain use memory and agents compared to CrewAI?"
final_answer = full_query_decomposition_rag_pipeline(query)
print("✅ Final Answer:\n")
print(final_answer)

✅ Final Answer:

Q: Here are sub-questions to decompose the complex question:
A: Please provide the complex question so I can help decompose it into sub-questions. 😊 

I'm ready to analyze the context you've given and break down the question into smaller, more manageable parts.  



Q: **What memory mechanisms does LangChain utilize in its applications?**
A: According to the text, LangChain uses **ConversationBufferMemory** and **ConversationSummaryMemory** as memory mechanisms. 


These modules allow the LLM to:

* **Maintain awareness of previous conversation turns** (ConversationBufferMemory)
* **Summarize long interactions to fit within token limits** (ConversationSummaryMemory) 


Q: **How do LangChain agents leverage memory to perform tasks?**
A: According to the provided text, LangChain agents use **context-aware memory** across steps.  

This means they can remember information from previous interactions in a task and use that context to inform their current decisions and actio

In [10]:
final_prompt = PromptTemplate.from_template("""
Given the following intermediate question and answer pairs, provide a concise final answer to the original question.
Intermediate Q&A:
{intermediate_qa}""")
final_chain = final_prompt | llm | StrOutputParser()
final_response = final_chain.invoke({"intermediate_qa": final_answer})
print("\n✅ Final Response:\n")
print(final_response)


✅ Final Response:

LangChain and CrewAI are both frameworks for building AI agents, but they differ in their core focus. LangChain specializes in agent planning and execution, utilizing context-aware memory and a planner-executor model. It offers features like hybrid retrieval and traceability. CrewAI, on the other hand, emphasizes role-based collaboration, facilitating communication and context sharing between specialized AI agents.  While CrewAI's specific memory mechanisms are not explicitly detailed, its strength lies in orchestrating complex tasks through collaborative workflows. 


Essentially, LangChain is geared towards individual agent intelligence and task completion, while CrewAI focuses on coordinating multiple agents towards a shared goal.

